In [1]:
# 查看 GPU
!nvidia-smi

# 查看预装 torch / torchvision 版本 (我们不改动它们)
import torch, torchvision, sys
print("torch:", torch.__version__, "cuda:", torch.version.cuda)
print("torchvision:", torchvision.__version__)


Thu Dec 18 00:16:39 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   49C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
# 克隆 InfLoRA 官方代码
!git clone https://github.com/liangyanshuo/InfLoRA.git
%cd InfLoRA
!ls -la


Cloning into 'InfLoRA'...
remote: Enumerating objects: 343, done.
remote: Counting objects: 100% (145/145), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 343 (delta 110), reused 87 (delta 87), pack-reused 198 (from 1)
Receiving objects: 100% (343/343), 7.83 MiB | 11.46 MiB/s, done.
Resolving deltas: 100% (157/157), done.
/content/InfLoRA
total 236
drwxr-xr-x 10 root root   4096 Dec 18 00:17 .
drwxr-xr-x  1 root root   4096 Dec 18 00:17 ..
drwxr-xr-x  2 root root   4096 Dec 18 00:17 configs
drwxr-xr-x  3 root root   4096 Dec 18 00:17 dataloaders
-rw-r--r--  1 root root  14388 Dec 18 00:17 environment.yaml
drwxr-xr-x  8 root root   4096 Dec 18 00:17 .git
-rw-r--r--  1 root root 161334 Dec 18 00:17 InfLoRA.png
-rw-r--r--  1 root root   1068 Dec 18 00:17 LICENSE
drwxr-xr-x  4 root root   4096 Dec 18 00:17 logs
-rw-r--r--  1 root root    895 Dec 18 00:17 main.py
drwxr-xr-x  3 root root   4096 Dec 18 00:17 methods
drwxr-xr-x  4 root root   4096 Dec 18 00:17 models
drwx

In [3]:
# 安装其它依赖（不安装 torch/torchvision）
!pip install -r requirements.txt 2>/dev/null || true

# 如果 repo 没有 requirements.txt，安装常见依赖：
!pip install timm==0.6.7 ftfy regex lmdb tqdm tensorboard

# 安装其它可能需要的小工具（如果 repo 在 utils 里调用）
!pip install einops


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.0/510.0 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.4/299.4 kB 29.7 MB/s eta 0:00:00
  Attempting uninstall: timm
    Found existing installation: timm 1.0.22
    Uninstalling timm-1.0.22:
      Successfully uninstalled timm-1.0.22


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# 创建存档路径（示例）
!mkdir -p /content/drive/MyDrive/inflora_logs


Mounted at /content/drive


In [ ]:
%%bash
# 确认 data 目录
!ls -la data || mkdir -p data && ls -la data

# 测试用 torchvision 下载 CIFAR100（如果 repo 自带自动下载可以跳过）
python - <<'PY'
from torchvision import datasets, transforms
datasets.CIFAR100(root='data', download=True)
print("CIFAR100 downloaded to data/")
PY


total 8
drwxr-xr-x  2 root root 4096 Dec 14 06:30 .
drwxr-xr-x 11 root root 4096 Dec 14 06:30 ..
CIFAR100 downloaded to data/


bash: line 2: !ls: command not found
100%|██████████| 169M/169M [00:13<00:00, 12.5MB/s]


In [4]:
!pip install ipdb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 37.0 MB/s eta 0:00:00


In [6]:
!pip list

Package                                  Version
---------------------------------------- -------------------
absl-py                                  1.4.0
accelerate                               1.12.0
access                                   1.1.9
affine                                   2.4.0
aiofiles                                 24.1.0
aiohappyeyeballs                         2.6.1
aiohttp                                  3.13.2
aiosignal                                1.4.0
aiosqlite                                0.21.0
alabaster                                1.0.0
albucore                                 0.0.24
albumentations                           2.0.8
ale-py                                   0.11.2
alembic                                  1.17.2
altair                                   5.5.0
annotated-types                          0.7.0
antlr4-python3-runtime                   4.9.3
anyio                                    4.12.0
anywidget                           

In [8]:
!pip freeze > requirements.txt


In [ ]:
# 把这个 cell 整体复制到 Colab 的 Python cell 里运行
import json, os
debug_cfg = {
    "prefix": "reproduce_debug",
    "dataset": "cifar100",
    "data_path": "data/",
    "memory_size": 0,
    "memory_per_class": 0,
    "fixed_memory": True,
    "shuffle": False,
    "init_cls": 10,
    "increment": 10,
    # 只跑两个 session（init + 1 increment）用于快速验证
    "total_sessions" : 2,
    "model_name": "InfLoRA",
    "net_type": "sip",
    "embd_dim" : 768,
    "num_heads": 12,
    "seed": [0],
    "EPSILON" : 1e-8,
    # 减小轮数以便快速通过 pipeline
    "init_epoch" : 2,
    "optim" : "adam",
    "init_lr" : 0.0005,
    "init_lr_decay" : 0.1,
    "init_weight_decay" : 0.0,
    "epochs" : 2,
    "lrate" : 0.0005,
    "lrate_decay" : 0.1,
    # 更小的 batch 以适应 Colab 显存
    "batch_size" : 32,
    "weight_decay" : 0.0,
    # 更小的 adapter rank，减少计算
    "rank": 4,
    "lamb": 0.95,
    "lame": 1.0,
    # 减少 workers
    "num_workers" : 2
}

os.makedirs("configs", exist_ok=True)
debug_path = "configs/cifar100_inflora_debug.json"
with open(debug_path, "w") as f:
    json.dump(debug_cfg, f, indent=2, ensure_ascii=False)

print("Wrote debug config to", debug_path)
print(open(debug_path).read())

# 运行命令（下面是两种 device 写法，按 main.py 支持的格式选其一）
print("\n--- 运行命令示例 ---")
print("方式 A (device 0):")
print("!python main.py --device 0 --config configs/cifar100_inflora_debug.json")
print("\n方式 B (device cuda:0):")
print("!python main.py --device cuda:0 --config configs/cifar100_inflora_debug.json")


Wrote debug config to configs/cifar100_inflora_debug.json
{
  "prefix": "reproduce_debug",
  "dataset": "cifar100",
  "data_path": "data/",
  "memory_size": 0,
  "memory_per_class": 0,
  "fixed_memory": true,
  "shuffle": false,
  "init_cls": 10,
  "increment": 10,
  "total_sessions": 2,
  "model_name": "InfLoRA",
  "net_type": "sip",
  "embd_dim": 768,
  "num_heads": 12,
  "seed": [
    0
  ],
  "EPSILON": 1e-08,
  "init_epoch": 2,
  "optim": "adam",
  "init_lr": 0.0005,
  "init_lr_decay": 0.1,
  "init_weight_decay": 0.0,
  "epochs": 2,
  "lrate": 0.0005,
  "lrate_decay": 0.1,
  "batch_size": 32,
  "weight_decay": 0.0,
  "rank": 4,
  "lamb": 0.95,
  "lame": 1.0,
  "num_workers": 2
}

--- 运行命令示例 ---
方式 A (device 0):
!python main.py --device 0 --config configs/cifar100_inflora_debug.json

方式 B (device cuda:0):
!python main.py --device cuda:0 --config configs/cifar100_inflora_debug.json


In [ ]:
!python main.py --device 0 --config configs/cifar100_inflora_debug.json

logs/cifar100/10_10_sip/InfLoRA/adam/4/0.95_1.0-0.0005/0
2025-12-14 06:40:55,219 [trainer.py] => config: configs/cifar100_inflora_debug.json
2025-12-14 06:40:55,219 [trainer.py] => device: [device(type='cuda', index=0)]
2025-12-14 06:40:55,219 [trainer.py] => prefix: reproduce_debug
2025-12-14 06:40:55,219 [trainer.py] => dataset: cifar100
2025-12-14 06:40:55,219 [trainer.py] => data_path: data/
2025-12-14 06:40:55,219 [trainer.py] => memory_size: 0
2025-12-14 06:40:55,219 [trainer.py] => memory_per_class: 0
2025-12-14 06:40:55,219 [trainer.py] => fixed_memory: True
2025-12-14 06:40:55,220 [trainer.py] => shuffle: False
2025-12-14 06:40:55,220 [trainer.py] => init_cls: 10
2025-12-14 06:40:55,220 [trainer.py] => increment: 10
2025-12-14 06:40:55,220 [trainer.py] => total_sessions: 2
2025-12-14 06:40:55,220 [trainer.py] => model_name: InfLoRA
2025-12-14 06:40:55,220 [trainer.py] => net_type: sip
2025-12-14 06:40:55,220 [trainer.py] => embd_dim: 768
2025-12-14 06:40:55,220 [trainer.py] =>

In [ ]:
%%bash
mkdir -p /content/drive/MyDrive/datasets/imagenet-r
cd /content/drive/MyDrive/datasets


In [ ]:
!pip install -q huggingface_hub


In [ ]:
%%bash
cd /content/drive/MyDrive/
wget https://people.eecs.berkeley.edu/~hendrycks/imagenet-r.tar
tar -xvf imagenet-r.tar



ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/IPython/core/magics/script.py", line 211, in shebang
    out, err = p.communicate(cell)
               ^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/subprocess.py", line 1209, in communicate
    stdout, stderr = self._communicate(input, endtime, timeout)
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/subprocess.py", line 2115, in _communicate
    ready = selector.select(timeout)
            ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/selectors.py", line 415, in select
    fd_event_list = self._selector.poll(timeout)
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File 

TypeError: object of type 'NoneType' has no len()

In [ ]:
%%bash
# 在 Colab 的 bash cell 中运行此脚本（确保当前目录为 /content/InfLoRA）
cp methods/inflora.py methods/inflora.py.bak2

python - <<'PY'
from pathlib import Path
import re

p = Path("methods/inflora.py")
s = p.read_text()

# loose pattern that matches the problematic expression using np.dot(... transpose())
pattern = r"activation\s*-\s*np\.dot\(\s*np\.dot\(\s*self\.feature_list\[i\]\s*,\s*self\.feature_list\[i\]\.transpose\(\s*\)\s*\)\s*,\s*activation\s*\)"

if re.search(pattern, s):
    replacement = (
        "        # Project activation using PyTorch (robust to numpy/torch types)\n"
        "        feat = self.feature_list[i]\n"
        "        # Ensure feat is a torch.Tensor\n"
        "        if not isinstance(feat, torch.Tensor):\n"
        "            feat = torch.as_tensor(feat)\n"
        "        # Ensure activation is a torch.Tensor on same device\n"
        "        if not isinstance(activation, torch.Tensor):\n"
        "            activation = torch.as_tensor(activation, device=feat.device)\n"
        "        else:\n"
        "            activation = activation.to(feat.device)\n"
        "        proj = feat @ feat.T @ activation\n"
        "        act_hat = activation - proj\n"
    )
    s_new = re.sub(pattern, replacement, s)
    p.write_text(s_new)
    print('Replaced matched pattern with PyTorch-safe code.')
else:
    # Try to catch a slightly different formatting: np.dot(..., self.feature_list[i].transpose())
    pattern2 = r"activation\s*-\s*np\.dot\(\s*self\.feature_list\[i\]\s*,\s*self\.feature_list\[i\]\.transpose\(\s*\)\s*\)\s*[,)]\s*activation?"
    if re.search(pattern2, s):
        # This replacement handles the form activation - np.dot(A, A.transpose()).dot(activation) or similar
        replacement2 = (
            "        # Project activation using PyTorch (robust to numpy/torch types)\n"
            "        feat = self.feature_list[i]\n"
            "        if not isinstance(feat, torch.Tensor):\n"
            "            feat = torch.as_tensor(feat)\n"
            "        if not isinstance(activation, torch.Tensor):\n"
            "            activation = torch.as_tensor(activation, device=feat.device)\n"
            "        else:\n"
            "            activation = activation.to(feat.device)\n"
            "        proj = feat @ feat.T @ activation\n"
            "        act_hat = activation - proj\n"
        )
        s_new = re.sub(pattern2, replacement2, s)
        p.write_text(s_new)
        print('Replaced alternate matched pattern with PyTorch-safe code.')
    else:
        print('No direct pattern found. Showing nearby lines for manual patching:\\n')
        # Print a few lines around likely area (search for "update_DualGPM" function)
        idx = s.find("def update_DualGPM")
        if idx != -1:
            start = max(0, idx - 400)
            end = min(len(s), idx + 1400)
            print(s[start:end])
        else:
            print("Could not find update_DualGPM function automatically. Please open methods/inflora.py to patch manually.")

# show whether any np.dot(...) or .transpose() usages remain in the file
remains_dot = re.findall(r"np\.dot\(", s_new if 's_new' in locals() else s)
remains_trans = re.findall(r"\.transpose\(", s_new if 's_new' in locals() else s)
print(f"Remaining np.dot occurrences in file: {len(remains_dot)}")
print(f"Remaining .transpose occurrences in file: {len(remains_trans)}")

# print a short excerpt around the first remaining .transpose if any
if len(remains_trans) > 0:
    loc = (s_new if 's_new' in locals() else s).find(".transpose(")
    excerpt_start = max(0, loc-120)
    excerpt_end = min(len(s), loc+120)
    print("Excerpt around first .transpose occurrence:\\n")
    print((s_new if 's_new' in locals() else s)[excerpt_start:excerpt_end])
PY


Replaced matched pattern with PyTorch-safe code.
Remaining np.dot occurrences in file: 4
Remaining .transpose occurrences in file: 2
Excerpt around first .transpose occurrence:\n
 # Projected Representation (Eq-8)
                    act_hat = np.dot(np.dot(self.feature_list[i],self.feature_list[i].transpose()),activation)
                    U,S,Vh = np.linalg.svd(act_hat, full_matrices=False)
                    #


In [ ]:
%%bash
# 备份并自动修补 methods/inflora.py（在 /content/InfLoRA 目录下运行）
cp methods/inflora.py methods/inflora.py.bak3

python - <<'PY'
from pathlib import Path
import re

p = Path("methods/inflora.py")
s = p.read_text()

# Ensure torch is imported: if not, add `import torch` near top (after other imports)
if "import torch" not in s:
    # try to insert after the first block of imports
    lines = s.splitlines()
    insert_at = 0
    for i, line in enumerate(lines[:60]):
        if line.strip().startswith("import") or line.strip().startswith("from"):
            insert_at = i+1
    lines.insert(insert_at, "import torch")
    s = "\n".join(lines)
    print("Inserted 'import torch' near the top of methods/inflora.py")

# 1) Replace .transpose() -> .T (safe for numpy and torch)
s = s.replace(".transpose()", ".T")
print("Replaced '.transpose()' -> '.T' (global)")

# 2) Replace nested np.dot(...) pattern that projects activation:
# pattern covers forms like:
# act_hat = np.dot(np.dot(self.feature_list[i],self.feature_list[i].T),activation)
# or with spaces
pattern = re.compile(
    r"act_hat\s*=\s*np\.dot\(\s*np\.dot\(\s*(self\.feature_list\[(?P<idx>[^\]]+)\])\s*,\s*(self\.feature_list\[\s*(?P=idx)\s*\]\.T)\s*\)\s*,\s*(?P<act>\w+)\s*\)"
)
def repl_proj(m):
    idx = m.group('idx')
    act = m.group('act')
    code = (
        f"        # Projected Representation (converted to PyTorch operations)\n"
        f"        feat = {m.group(1)}\n"
        f"        # Ensure feat is a torch.Tensor on device\n"
        f"        if not isinstance(feat, torch.Tensor):\n"
        f"            feat = torch.as_tensor(feat)\n"
        f"        # Ensure activation is a torch.Tensor on same device\n"
        f"        if not isinstance({act}, torch.Tensor):\n"
        f"            {act} = torch.as_tensor({act}, device=feat.device)\n"
        f"        else:\n"
        f"            {act} = {act}.to(feat.device)\n"
        f"        proj = feat @ feat.T @ {act}\n"
        f"        act_hat = {act} - proj\n"
    )
    return code

s_new = pattern.sub(repl_proj, s)
if s_new != s:
    s = s_new
    print("Replaced nested np.dot(... .T ...) projection pattern with PyTorch-safe code.")
else:
    print("No nested np.dot projection pattern found by regex. (Maybe different formatting)")

# 3) Replace any remaining simpler pattern variants like:
# act_hat = np.dot(self.feature_list[i] @ self.feature_list[i].T, activation) or similar
# We'll search for occurrences of "np.dot(" with feature_list and replace with explicit tensor ops near them.
simple_pattern = re.compile(
    r"(?P<indent>\s*)act_hat\s*=\s*np\.dot\(\s*(self\.feature_list\[(?P<idx2>[^\]]+)\]\.T?\s*|\s*self\.feature_list\[(?P=idx2)\])\s*,\s*(?P<act2>\w+)\s*\)"
)
def repl_simple(m):
    idx = m.group('idx2')
    act = m.group('act2')
    indent = m.group('indent')
    code = (
        f"{indent}# Converted simple np.dot projection to PyTorch\n"
        f"{indent}feat = self.feature_list[{idx}]\n"
        f"{indent}if not isinstance(feat, torch.Tensor):\n"
        f"{indent}    feat = torch.as_tensor(feat)\n"
        f"{indent}if not isinstance({act}, torch.Tensor):\n"
        f"{indent}    {act} = torch.as_tensor({act}, device=feat.device)\n"
        f"{indent}else:\n"
        f"{indent}    {act} = {act}.to(feat.device)\n"
        f"{indent}proj = feat @ feat.T @ {act}\n"
        f"{indent}act_hat = {act} - proj\n"
    )
    return code

s_new = simple_pattern.sub(repl_simple, s)
if s_new != s:
    s = s_new
    print("Replaced simpler np.dot projection occurrences.")
else:
    print("No simpler np.dot projection occurrences replaced by this pass.")

# 4) Replace np.linalg.svd(...) -> torch.linalg.svd(...), with tensor guard
svd_pattern = re.compile(r"(?P<indent>\s*)U\s*,\s*S\s*,\s*Vh\s*=\s*np\.linalg\.svd\(\s*(?P<arg>\w+)\s*,\s*full_matrices\s*=\s*False\s*\)")
def repl_svd(m):
    indent = m.group('indent')
    arg = m.group('arg')
    code = (
        f"{indent}# Use PyTorch SVD for tensor compatibility\n"
        f"{indent}if not isinstance({arg}, torch.Tensor):\n"
        f"{indent}    {arg} = torch.as_tensor({arg})\n"
        f"{indent}# Try moving to same device as feat if feat exists\n"
        f"{indent}try:\n"
        f"{indent}    {arg} = {arg}.to(feat.device)\n"
        f"{indent}except Exception:\n"
        f"{indent}    pass\n"
        f"{indent}U, S, Vh = torch.linalg.svd({arg}, full_matrices=False)\n"
    )
    return code

s_new = svd_pattern.sub(repl_svd, s)
if s_new != s:
    s = s_new
    print("Replaced np.linalg.svd(...) with torch.linalg.svd(...) and added tensor guards.")
else:
    print("No exact np.linalg.svd(...) pattern found for replacement. Trying broader match...")

# broader match for np.linalg.svd occurrences
if "np.linalg.svd" in s:
    s = s.replace("np.linalg.svd(", "torch.linalg.svd(")
    print("Replaced remaining 'np.linalg.svd(' -> 'torch.linalg.svd(' (note: may need device guards).")

# write back
p.write_text(s)
print("Wrote patched methods/inflora.py")

# Report remaining np.dot and .transpose occurrences for manual review
remains_dot = len(re.findall(r"np\.dot\(", s))
remains_transpose_calls = len(re.findall(r"\.transpose\(", s))
remains_T_attrs = len(re.findall(r"\.T\b", s))  # occurrences of .T
print(f"Remaining np.dot occurrences: {remains_dot}")
print(f"Remaining .transpose(...) calls: {remains_transpose_calls}")
print(f"Occurrences of .T in file (should exist where .transpose replaced): {remains_T_attrs}")

# show excerpt around first occurrence of torch.linalg.svd if any
loc = s.find("torch.linalg.svd")
if loc != -1:
    start = max(0, loc-120)
    end = min(len(s), loc+240)
    print("\\nExcerpt around first torch.linalg.svd occurrence:\\n")
    print(s[start:end])
else:
    print("\\nNo torch.linalg.svd occurrences found in file after replacement.")
PY


Replaced '.transpose()' -> '.T' (global)
Replaced nested np.dot(... .T ...) projection pattern with PyTorch-safe code.
No simpler np.dot projection occurrences replaced by this pass.
Replaced np.linalg.svd(...) with torch.linalg.svd(...) and added tensor guards.
Replaced remaining 'np.linalg.svd(' -> 'torch.linalg.svd(' (note: may need device guards).
Wrote patched methods/inflora.py
Remaining np.dot occurrences: 2
Remaining .transpose(...) calls: 0
Occurrences of .T in file (should exist where .transpose replaced): 11
\nExcerpt around first torch.linalg.svd occurrence:\n
tance(module, Attention_LoRA):
                        cur_matrix = module.cur_matrix
                        U, S, V = torch.linalg.svd(cur_matrix)
                        module.lora_A_k[self._cur_task].weight.data.copy_(U[:,:module.rank].T/math.sqrt(3))
                        module.lora_A_v[self._cur_task].weight.data.copy_(U[:,:module.rank].T/math.sqrt


In [ ]:
%%bash
# 在仓库根目录 /content/InfLoRA 下运行
cp methods/inflora.py methods/inflora.py.bak4

python - <<'PY'
from pathlib import Path
import re
p = Path("methods/inflora.py")
s = p.read_text()

# Ensure numpy and torch imports exist
if "import numpy as np" not in s:
    # Try to insert near imports
    lines = s.splitlines()
    insert_at = 0
    for i, line in enumerate(lines[:80]):
        if line.strip().startswith("import") or line.strip().startswith("from"):
            insert_at = i+1
    lines.insert(insert_at, "import numpy as np")
    s = "\n".join(lines)
    print("Inserted 'import numpy as np' near top of file.")

if "import torch" not in s:
    lines = s.splitlines()
    insert_at = 0
    for i, line in enumerate(lines[:80]):
        if line.strip().startswith("import") or line.strip().startswith("from"):
            insert_at = i+1
    lines.insert(insert_at, "import torch")
    s = "\n".join(lines)
    print("Inserted 'import torch' near top of file.")

# Insert helper function _matmul(a,b) if not present
if "_matmul(" not in s:
    helper = """
# Helper: safe matrix multiply that accepts numpy arrays or torch tensors and returns a torch.Tensor on correct device
def _matmul(a, b):
    # Prefer torch tensors for computation
    # If inputs are numpy arrays, convert them. If inputs are torch tensors, ensure they are on same device.
    if isinstance(a, np.ndarray):
        a = torch.as_tensor(a)
    if isinstance(b, np.ndarray):
        # If 'a' is already a tensor, put 'b' on same device; else let torch decide
        try:
            b = torch.as_tensor(b, device=a.device)
        except Exception:
            b = torch.as_tensor(b)
    # Finally ensure both are torch tensors
    if not isinstance(a, torch.Tensor):
        a = torch.as_tensor(a)
    if not isinstance(b, torch.Tensor):
        # place b on same device as a if possible
        try:
            b = torch.as_tensor(b, device=a.device)
        except Exception:
            b = torch.as_tensor(b)
    # Use @ for matrix multiplication (works for 1D/2D appropriately like np.dot)
    return a @ b

"""
    # insert helper after imports block (find last import line)
    lines = s.splitlines()
    last_import_idx = 0
    for i, line in enumerate(lines[:200]):
        if line.strip().startswith("import") or line.strip().startswith("from"):
            last_import_idx = i
    # insert after a small offset
    insert_pos = last_import_idx + 1
    lines.insert(insert_pos, helper)
    s = "\n".join(lines)
    print("Inserted helper function _matmul into methods/inflora.py")

# Replace all occurrences of np.dot( with _matmul(
count_before = len(re.findall(r"np\.dot\(", s))
s_replaced = s.replace("np.dot(", "_matmul(")
count_after = len(re.findall(r"np\.dot\(", s_replaced))
replacements = count_before - count_after

p.write_text(s_replaced)
print(f"Replaced {replacements} occurrences of 'np.dot(' with '_matmul(' (before: {count_before}, after: {count_after})")

# Report remaining np.dot occurrences and show contexts
remains_dot = list(re.finditer(r"_matmul\(", s_replaced))
print(f"Occurrences of new '_matmul(' in file: {len(remains_dot)}")
for i, m in enumerate(remains_dot[:10]):
    start = m.start()
    excerpt_start = max(0, start-120)
    excerpt_end = min(len(s_replaced), start+120)
    print(f"--- occurrence {i+1} context ---")
    print(s_replaced[excerpt_start:excerpt_end])
    print("----------------------------")

# Also show if any np.dot remains
rem_npdot = len(re.findall(r"np\.dot\(", s_replaced))
print("Remaining literal 'np.dot(' occurrences after replacement:", rem_npdot)

# Quick check: any .transpose( left?
rem_transpose = len(re.findall(r"\.transpose\(", s_replaced))
print("Remaining '.transpose(' calls:", rem_transpose)

# Save final file (already written)
print("Patched file written to methods/inflora.py; backup at methods/inflora.py.bak4")
PY


Inserted helper function _matmul into methods/inflora.py
Replaced 2 occurrences of 'np.dot(' with '_matmul(' (before: 2, after: 0)
Occurrences of new '_matmul(' in file: 3
--- occurrence 1 context ---
elper: safe matrix multiply that accepts numpy arrays or torch tensors and returns a torch.Tensor on correct device
def _matmul(a, b):
    # Prefer torch tensors for computation
    # If inputs are numpy arrays, convert them. If inputs are 
----------------------------
--- occurrence 2 context ---
               # update GPM by Projected Representation (Eq-8)
                    act_feature = self.feature_list[i] - _matmul(_matmul(U[:,0:r],U[:,0:r].T),self.feature_list[i])
                    Ui, Si, Vi = torch.linalg.svd(act_feature
----------------------------
--- occurrence 3 context ---
       # update GPM by Projected Representation (Eq-8)
                    act_feature = self.feature_list[i] - _matmul(_matmul(U[:,0:r],U[:,0:r].T),self.feature_list[i])
                    Ui, Si, Vi

In [ ]:
!python main.py --device 0 --config configs/cifar100_inflora_debug.json 2>&1 | tee /content/inflora_debug_run.log

Traceback (most recent call last):
  File "/content/InfLoRA/main.py", line 3, in <module>
    from trainer import train
  File "/content/InfLoRA/trainer.py", line 8, in <module>
    from utils import factory
  File "/content/InfLoRA/utils/factory.py", line 4, in <module>
    from methods.inflora import InfLoRA
  File "/content/InfLoRA/methods/inflora.py", line 389
    act_hat =         # Project activation using PyTorch (robust to numpy/torch types)
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
SyntaxError: invalid syntax


In [ ]:
%%bash
cp methods/inflora.py methods/inflora.py.bak_fix_syntax

python - <<'PY'
import re
from pathlib import Path

path = Path("methods/inflora.py")
code = path.read_text()

# ---------------------------------------------------------
# PATCH 1: 修复 update_DualGPM 的 act_hat block
# ---------------------------------------------------------
dual_block_pattern = r"""
act_hat\s*=.*?  #\s*Project activation[\s\S]*?  # 注意，这里的注释 block 有可能被损坏
"""

# 用一个严格正确的版本替换
dual_block_replacement = """
        # Project activation using PyTorch (robust to numpy/torch types)
        feat = self.feature_list[i]
        if isinstance(feat, np.ndarray):
            feat = torch.as_tensor(feat)

        if not isinstance(activation, torch.Tensor):
            activation = torch.as_tensor(activation, device=feat.device)
        else:
            activation = activation.to(feat.device)

        proj = _matmul(_matmul(feat, feat.T), activation)
        act_hat = activation - proj
"""

# 宽松替换：匹配行首有 act_hat = 的投影段
code = re.sub(r"act_hat\s*=.*", dual_block_replacement, code)


# ---------------------------------------------------------
# PATCH 2: 修复 update_GPM 中 act_feature 块
# ---------------------------------------------------------

gpm_block_pattern = r"act_feature\s*=.*"
gpm_block_replacement = """
                    # Projected Representation (Eq-8)
                    act_feature = self.feature_list[i] - _matmul(_matmul(U[:, :r], U[:, :r].T), self.feature_list[i])
"""

code = re.sub(gpm_block_pattern, gpm_block_replacement, code)


# 写回文件
path.write_text(code)
print("Patched syntax and projection blocks successfully.")
print("Backup: methods/inflora.py.bak_fix_syntax")
PY


Patched syntax and projection blocks successfully.
Backup: methods/inflora.py.bak_fix_syntax


In [ ]:
!python main.py --device 0 --config configs/cifar100_inflora_debug.json 2>&1 | tee /content/inflora_debug_run.log

Traceback (most recent call last):
  File "/content/InfLoRA/main.py", line 3, in <module>
    from trainer import train
  File "/content/InfLoRA/trainer.py", line 8, in <module>
    from utils import factory
  File "/content/InfLoRA/utils/factory.py", line 4, in <module>
    from methods.inflora import InfLoRA
  File "/content/InfLoRA/methods/inflora.py", line 431
    if not isinstance(act_hat, torch.Tensor):
IndentationError: unexpected indent


In [ ]:
%%bash
# 备份并修复 methods/inflora.py 的 act_hat/proj 区块（在 /content/InfLoRA 下运行）
cp methods/inflora.py methods/inflora.py.bak5

python - <<'PY'
from pathlib import Path
import re
p = Path("methods/inflora.py")
s = p.read_text()

# We'll replace any block that includes 'proj = ' and 'act_hat =' (spanning multiple lines)
# with a correctly indented, robust implementation.
pattern = re.compile(
    r"(?P<indent>[\t ]*)(?:#.*\n\s*)*proj\s*=.*?\n\s*act_hat\s*=.*?(?:\n|$)",
    re.S
)

replacement_template = (
    "{indent}# Compute projection and ensure tensor/device consistency\n"
    "{indent}proj = _matmul(_matmul(feat, feat.T), activation)\n"
    "{indent}act_hat = activation - proj\n"
    "{indent}# Ensure act_hat is a torch.Tensor on the same device as feat\n"
    "{indent}if not isinstance(act_hat, torch.Tensor):\n"
    "{indent}    act_hat = torch.as_tensor(act_hat, device=feat.device)\n"
    "{indent}else:\n"
    "{indent}    act_hat = act_hat.to(feat.device)\n"
)

replacements = 0
def repl(m):
    global replacements
    indent = m.group('indent') or ""
    replacements += 1
    return replacement_template.format(indent=indent)

s_new = pattern.sub(repl, s)

if replacements == 0:
    # If no proj/act_hat block matched, try to match 'act_hat =' alone and replace surrounding lines conservatively
    pattern2 = re.compile(r"(?P<indent>[\t ]*)act_hat\s*=.*?(?:\n|$)", re.S)
    s_new2 = pattern2.sub(lambda m: replacement_template.format(indent=m.group('indent') or ""), s)
    if s_new2 != s:
        s_new = s_new2
        replacements = len(re.findall(r"act_hat\s*=", s_new))
        print(f"No proj-block matches; replaced {replacements} 'act_hat =' occurrences with canonical block.")
    else:
        print("No obvious 'proj'/'act_hat' blocks found by the script. Showing a short excerpt around line 400-460 for manual inspection:")
        start = 3800 if len(s) > 3800 else 0
        print(s[start:start+1500])
        # write no-change file (keep backup only)
        p.write_text(s)
        raise SystemExit("No automatic replacements performed.")

# Write patched file
p.write_text(s_new)
print(f"Patched {replacements} proj/act_hat block(s) in methods/inflora.py (backup at methods/inflora.py.bak5).")

# Show context around first occurrence for verification
loc = s_new.find("act_hat")
if loc != -1:
    start = max(0, loc-200)
    end = min(len(s_new), loc+400)
    print("\\nExcerpt around first 'act_hat' after patch:\\n")
    print(s_new[start:end])
else:
    print("No 'act_hat' found after patch (unexpected).")
PY


Patched 1 proj/act_hat block(s) in methods/inflora.py (backup at methods/inflora.py.bak5).
\nExcerpt around first 'act_hat' after patch:\n
m copy import deepcopy
from utils.schedulers import CosineSchedule
import ipdb
import math

# Compute projection and ensure tensor/device consistency
proj = _matmul(_matmul(feat, feat.T), activation)
act_hat = activation - proj
# Ensure act_hat is a torch.Tensor on the same device as feat
if not isinstance(act_hat, torch.Tensor):
    act_hat = torch.as_tensor(act_hat, device=feat.device)
else:
    act_hat = act_hat.to(feat.device)



                except Exception:


                    pass


                U, S, Vh = torch.linalg.svd(act_hat, full_matrices=False)

                # criter


In [ ]:
!python main.py --device 0 --config configs/cifar100_inflora_debug.json 2>&1 | tee /content/inflora_debug_run.log


Traceback (most recent call last):
  File "/content/InfLoRA/main.py", line 3, in <module>
    from trainer import train
  File "/content/InfLoRA/trainer.py", line 8, in <module>
    from utils import factory
  File "/content/InfLoRA/utils/factory.py", line 4, in <module>
    from methods.inflora import InfLoRA
  File "/content/InfLoRA/methods/inflora.py", line 32
    except Exception:
IndentationError: unexpected indent


In [ ]:
%%bash
# 确保在仓库根目录
pwd
ls -la methods/inflora.py*
echo "---- git status (to see uncommitted changes) ----"
git status --porcelain || true


/content/InfLoRA
-rw-r--r-- 1 root root  2154 Dec  4 08:46 methods/inflora.py
-rw-r--r-- 1 root root 21614 Dec  4 08:14 methods/inflora.py.bak
-rw-r--r-- 1 root root 21586 Dec  4 08:39 methods/inflora.py.bak2
-rw-r--r-- 1 root root 22536 Dec  4 08:41 methods/inflora.py.bak3
-rw-r--r-- 1 root root 25078 Dec  4 08:43 methods/inflora.py.bak4
-rw-r--r-- 1 root root 31023 Dec  4 08:46 methods/inflora.py.bak5
-rw-r--r-- 1 root root 26111 Dec  4 08:45 methods/inflora.py.bak_fix_syntax
---- git status (to see uncommitted changes) ----
 M logs/cifar100/10_10_sip/InfLoRA/adam/10/0.95_1.0-0.0005/0.log
 M methods/inflora.py
?? __pycache__/trainer.cpython-312.pyc
?? configs/cifar100_inflora_debug.json
?? data/
?? logs/cifar100/10_10_sip/InfLoRA/adam/4/
?? methods/__pycache__/__init__.cpython-312.pyc
?? methods/__pycache__/base.cpython-312.pyc
?? methods/__pycache__/inflora.cpython-312.pyc
?? methods/__pycache__/inflora_ca.cpython-312.pyc
?? methods/__pycache__/inflora_ca1.cpython-312.pyc
?? methods

In [ ]:
%%bash
# 示例：如果存在 methods/inflora.py.bak（最早的备份），用它覆盖当前文件
cp methods/inflora.py.bak methods/inflora.py
echo "Restored from methods/inflora.py.bak"

# 如果没有 .bak 而只有 .bak2/.bak3，可以逐一尝试
# cp methods/inflora.py.bak2 methods/inflora.py
# cp methods/inflora.py.bak3 methods/inflora.py


Restored from methods/inflora.py.bak


In [ ]:
!python main.py --device 0 --config configs/cifar100_inflora_debug.json

logs/cifar100/10_10_sip/InfLoRA/adam/4/0.95_1.0-0.0005/0
2025-12-04 09:04:33,478 [trainer.py] => config: configs/cifar100_inflora_debug.json
2025-12-04 09:04:33,478 [trainer.py] => device: [device(type='cuda', index=0)]
2025-12-04 09:04:33,478 [trainer.py] => prefix: reproduce_debug
2025-12-04 09:04:33,478 [trainer.py] => dataset: cifar100
2025-12-04 09:04:33,478 [trainer.py] => data_path: data/
2025-12-04 09:04:33,478 [trainer.py] => memory_size: 0
2025-12-04 09:04:33,478 [trainer.py] => memory_per_class: 0
2025-12-04 09:04:33,479 [trainer.py] => fixed_memory: True
2025-12-04 09:04:33,479 [trainer.py] => shuffle: False
2025-12-04 09:04:33,479 [trainer.py] => init_cls: 10
2025-12-04 09:04:33,479 [trainer.py] => increment: 10
2025-12-04 09:04:33,479 [trainer.py] => total_sessions: 2
2025-12-04 09:04:33,479 [trainer.py] => model_name: InfLoRA
2025-12-04 09:04:33,479 [trainer.py] => net_type: sip
2025-12-04 09:04:33,479 [trainer.py] => embd_dim: 768
2025-12-04 09:04:33,479 [trainer.py] =>

In [ ]:

!python main.py --device 0 --config configs/cifar100_inflora_debug.json


logs/cifar100/10_10_sip/InfLoRA/adam/4/0.95_1.0-0.0005/0
2025-12-04 09:26:42,044 [trainer.py] => config: configs/cifar100_inflora_debug.json
2025-12-04 09:26:42,044 [trainer.py] => device: [device(type='cuda', index=0)]
2025-12-04 09:26:42,044 [trainer.py] => prefix: reproduce_debug
2025-12-04 09:26:42,044 [trainer.py] => dataset: cifar100
2025-12-04 09:26:42,044 [trainer.py] => data_path: data/
2025-12-04 09:26:42,044 [trainer.py] => memory_size: 0
2025-12-04 09:26:42,044 [trainer.py] => memory_per_class: 0
2025-12-04 09:26:42,044 [trainer.py] => fixed_memory: True
2025-12-04 09:26:42,044 [trainer.py] => shuffle: False
2025-12-04 09:26:42,044 [trainer.py] => init_cls: 10
2025-12-04 09:26:42,045 [trainer.py] => increment: 10
2025-12-04 09:26:42,045 [trainer.py] => total_sessions: 2
2025-12-04 09:26:42,045 [trainer.py] => model_name: InfLoRA
2025-12-04 09:26:42,045 [trainer.py] => net_type: sip
2025-12-04 09:26:42,045 [trainer.py] => embd_dim: 768
2025-12-04 09:26:42,045 [trainer.py] =>

In [ ]:
print("hello")

hello
